In [ ]:
# !pip install keras_ocr
import math
import matplotlib.pyplot as plt
import keras_ocr
import numpy as np
import cv2
pipeline = keras_ocr.pipeline.Pipeline()#read image from the an image path (a jpg/png file or an image url)
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)
def inpaint_text(img_path, pipeline):
    # read image
    img = keras_ocr.tools.read(img_path)
    # generate (word, box) tuples
    words=[]
    boxes=[]
    prediction_groups = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:
           x0, y0 = box[1][0]
           x1, y1 = box[1][1]
           x2, y2 = box[1][2]
           x3, y3 = box[1][3]
        #    box[1].tolist()
        #    box[1].to(list)
           words.append(box[0])
           boxes.append(box[1].tolist())
           x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
           x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)

           thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))

           cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,
           thickness)
           img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
           img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
           cv2.imwrite(f'removed_text_data/{img_path}',img_rgb)

    return(img_rgb,words,boxes,prediction_groups,mask)

In [ ]:
import cv2
import json
# !cp -r drive/MyDrive/data1/img data/img

text_removed={}
i=int(0)
with open("../data/train.jsonl", 'r') as f:
    for line in f:
        entry = json.loads(line)
        if (i==100):
            with open("text_removed.json", 'w') as f:
                  json.dump(text_removed, f)
            i=0
        else:
            i+=1
    # Iterate through each image entry in the JSON data
        # print(entry)
        # image_id = int(entry['id'])
        image_path = "../data/"+entry['img']
        # label = int(entry['label'])
        # text = entry['text']
        # image_path = "drive"
        imgs,words,boxes,prediction_group,mask=inpaint_text(image_path,pipeline)
        text_removed[int(entry['id'])]={
            'id' : int(entry['id']),
            'words' : words,
            'boxes' : boxes,
            # 'prediction_group' : prediction_group,
            # 'mask' : mask,
        }


In [11]:
from ultralytics import YOLO
import cv2
import time
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = YOLO('yolov8l.pt')
model.to(device)
object_detection_results={}
import json
with open("object_details.json","r") as f:
    object_detection_results=json.load(f)
with open ("text_removed.json","r") as f:
    text_removed=json.load(f)
object_detection_temp={}
for key,elem in text_removed.items():
    # print(elem['id'])
    elem['id']=str(elem['id'])
    if (str(elem['id']).__len__()==4):
        elem['id']="0"+elem['id']
    results = model(f"data/img/{elem['id']}.png")
    
    class_ids=[]
    confs=[]
    bboxs=[]
       
    for result in results:
        for conf,class_id in zip(result.boxes.conf,result.boxes.cls):
            if conf > 0.5:
                class_ids.append(int(class_id))
                confs.append(float(conf))
    object_detection_temp[elem['id']]={
        'class_ids': class_ids,
        'confs': confs,
    }



image 1/1 /home/ansh/Documents/Precog/Kushal/data/img/42953.png: 640x448 2 persons, 1 tie, 20.0ms
Speed: 1.8ms preprocess, 20.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 /home/ansh/Documents/Precog/Kushal/data/img/23058.png: 448x640 2 persons, 1 tie, 22.0ms
Speed: 1.7ms preprocess, 22.0ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/ansh/Documents/Precog/Kushal/data/img/13894.png: 640x448 2 cats, 2 ties, 20.3ms
Speed: 1.8ms preprocess, 20.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 /home/ansh/Documents/Precog/Kushal/data/img/37408.png: 640x576 2 dogs, 27.4ms
Speed: 2.0ms preprocess, 27.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 576)

image 1/1 /home/ansh/Documents/Precog/Kushal/data/img/82403.png: 512x640 1 person, 1 tie, 23.2ms
Speed: 1.6ms preprocess, 23.2ms inference, 0.8ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/ansh/Documents/P

In [13]:
text_extracted=0
non_extracted=0
conf_text=0
conf_non=0
for key,elem in object_detection_temp.items():
    elem2=object_detection_results.get(key)
    if elem2==None:
        text_extracted+=1
        continue
    if (len(elem['class_ids'])>len(elem2['class_ids'])):
        text_extracted+=1
    elif (len(elem['class_ids'])<len(elem2['class_ids'])):
        non_extracted+=1
    sum_conf_text=sum(elem['confs'])
    sum_conf_non=sum(elem2['confs'])
    if (sum_conf_non>sum_conf_text):
        conf_non+=1
    elif (sum_conf_non<sum_conf_text):
        conf_non+=1
        
print(conf_text,conf_non)
print(text_extracted,text_removed)

0 1933
229 {'42953': {'id': '42953', 'words': ['its', 'their', 'character', 'their', 'color', 'not', 'that', 'matters'], 'boxes': [[[7.0, 13.0], [45.0, 13.0], [45.0, 40.0], [7.0, 40.0]], [[46.46985626220703, 14.565488815307617], [118.74947357177734, 12.233888626098633], [119.6153793334961, 39.07691955566406], [47.33576202392578, 41.40852355957031]], [[122.0, 14.0], [261.0, 14.0], [261.0, 40.0], [122.0, 40.0]], [[89.0, 353.0], [171.0, 353.0], [171.0, 378.0], [89.0, 378.0]], [[176.0, 354.0], [263.0, 354.0], [263.0, 378.0], [176.0, 378.0]], [[30.089662551879883, 357.524169921875], [85.72069549560547, 354.25177001953125], [86.98621368408203, 375.76556396484375], [31.355178833007812, 379.0379638671875]], [[43.0, 376.0], [109.0, 376.0], [109.0, 399.0], [43.0, 399.0]], [[113.0, 376.0], [238.0, 376.0], [238.0, 399.0], [113.0, 399.0]]]}, '23058': {'id': '23058', 'words': ['it', 'afraid', 'be', 'iove', 'don', 'to', 'again', 'is', 'like', 'not', 'everyone', 'your', 'ex'], 'boxes': [[[236.0, 19.0]

In [14]:
with open("detection_temp.json", 'w') as f:
    json.dump(object_detection_temp, f)